# 3D examples to post-process the pressure for incompressible flow

In [ ]:
from trustutils import run 

run.introduction("MR")

run.TRUST_parameters("1.7.6")

### Description: 

This validation form aims to show how to post-process the pressure field with TRUST. The idea is to know what is coded behind the keywords: pression, pression_Pa, pressure, P_star, pression_hydrostatique on a case for which we know the analytical solution. At the end of this report, we will show you how to post-process the total pressure.

In [ ]:
from trustutils import run 

for i in ["g", "gx"]:
    for j in ["muscl", "negligeable", "ef_stab"]:
        run.addCase(f"PAR_VEF_P0P1_{i}_{j}",f"PAR_VEF_P0P1_{i}_{j}.data",nbProcs=3)
        
run.printCases()
#run.runCases()

## General information

### Poiseuille flow

Consider a laminar flow ($Re<2000$) of a fluid of viscosity $\mu$ and density $\rho$, in a cylindrical pipe of radius $R=0.1$, placed horizontally. The figure below highlights the configuration and makes it possible to define a cylindrical reference mark whose axis of revolution is that of the pipe and corresponds to the direction of the laminar flow. 

<img src="src/fig.png">

Under these conditions, since the flow is laminar, the velocity vector at every point is written: 

$$\vec{V}=v_r \vec{e}_r + v_{\theta} \vec{e}_{\theta} + v_x\vec{e}_x=u(r,\theta,x) \vec{e}_x$$ 

as here $v_r=v_{\theta}=0$. 

Then in cylindrical coordinates: 

$$ \nabla \cdot \vec{V} = 0 = \frac{1}{r} \frac{\partial (r v_r)}{\partial r} + \frac{1}{r} \frac{\partial v_{\theta}}{\partial \theta} + \frac{\partial v_x}{\partial x} = \frac{\partial u}{\partial x}=0$$ 

So $\vec{V}=u(r,\theta) \vec{e}_x$. And by symmetry of revolution $\frac{\partial v_{\theta}}{\partial \theta} =0$ so: 

$$ \vec{V}=u(r) \vec{e}_x$$ 

Now, take the equation governing the evolution of the total pressure $P_t(=p+\rho g z + \frac{1}{2}\rho v^2)$ along a laminar flow oriented along the axis $x$: 

$$\frac{dP_t}{dx}=\mu \Delta u$$  

And develop the expression of the Laplacian taking into account the fact that $u$ depends only on $x$: 

$$\frac{dP_t}{dx}=\mu \frac{1}{r} \frac{\partial}{\partial r} \left(r \frac{\partial u}{\partial r}\right)$$ 

Where the left-hand term depends only on $x$ and on the right-hand side only $r$. It follows that the members of this equation are equal to the same constant K. Thus, we have: 

$\displaystyle{\frac{dP_t}{dx}=K}$ reflecting the regular pressure drop;

$\displaystyle{\mu \frac{1}{r} \frac{\partial}{\partial r} \left(r \frac{\partial u}{\partial r}\right)}$ allowing the determination of the velocity profile.

Solving the second equation yields: 

$$u(r)=\frac{K}{4 \mu} r^2 + K^{'} ln(r) + K^{''}$$ 

Using $u(r=R)=0$ and $u(r=0)=u_{max}$ then: 

$$u(r)=\frac{K}{4 \mu} (r^2-R^2)$$ 

And we can write $u_{max}=u(0)=-\frac{KR^2}{4\mu}$ so 

$$K=-\frac{4 \mu}{R^2} u_{max}$$ 

The average velocity $V_{moy}=<u>$ can be expressed as a function of the volume flow rate $Q_v$: 

$$<u>=\frac{Q_v}{S}=\frac{1}{\pi R^2} \iint V \cdot \vec{n} dS = \frac{1}{\pi R^2} \iint u(r) dS=\frac{1}{\pi R^2} \int_0^R u(r) 2 \pi r dr= -\frac{KR^2}{8\mu}=-\frac{u_{max}}{2}$$ 

with $S=\pi R^2$ so $dS=2\pi r dr$. So $\displaystyle{Q_v=<u> S=<u>\pi R^2 =-\frac{K \pi R^4}{8 \mu}}$.

Then $\displaystyle{\frac{dP_t}{dx}=K}$ leads to: 

$$\Delta P_t =-\frac{dP_t}{dx} \Delta x=-K \Delta x= \frac{8 \mu <u> L}{R^2}$$ 

with $L=\Delta x$ the length of the pipe.

Set $D=2R$ the height of the pipe (its diameter) and $v=<u>$ the average velocity, then: 

$$\Delta P_t=\frac{8 \mu v L}{(D/2)^2}=\left(\frac{1}{2}\rho v^2\right)\left(\frac{64 \mu}{\rho v D}\right)\frac{L}{D} = \left(\frac{1}{2}\rho v^2\right)\left(\frac{64}{Re}\right)\frac{L}{D}$$

with $Re=\frac{\rho v D}{\mu}$ the Reynolds number where $\lambda=\frac{64}{Re}$ is the pressure loss coefficient.

Reference: http://res-nlp.univ-lemans.fr/NLP_C_M02_G02/co/Contenu_29.html

### Geometry used: VEF mesh

Here we perform calculation on a column of water. 

Inlet mesh done with gmsh, then we extrude it with TRUST (nb_elem_tot is 71232 elements).

In [ ]:
from trustutils import visit

a = visit.Show("PAR_VEF_P0P1_g_muscl/PAR_VEF_P0P1_g_muscl.lata",'Mesh','dom')
a.normal3D([0.6,-0.81,0.07])
a.up3D([-0.01,0.08,1])
a.plot()


### Probes localisation on Inlet and Outlet boundaries

The meshes are the same on the Inlet and Outlet. 



Coordinates of the <font color='red'>red</font> probe: 
(0, −3.35372631e−4, −2.89921075e−4)

Coordinates of the <font color='green'>green</font> probe: (0, -2.12652670e-2, -2.64793222e-2)


In [ ]:
from trustutils import visit

a = visit.Show("PAR_VEF_P0P1_g_muscl/PAR_VEF_P0P1_g_muscl_P_TOT1DRAWA.son","Mesh","MESH")
a.normal3D([-1,0.,0.])
a.up3D([0.,0.,1])
a.zoom3D([0.15,0.15,0.8])
a.meshColor("red")
a.visitCommand('MeshAtts.pointSizePixels=10')
a.visitCommand('SetPlotOptions(MeshAtts)')
a.addField("PAR_VEF_P0P1_g_muscl/PAR_VEF_P0P1_g_muscl_P_TOT2DRAWA.son","Mesh","MESH")
a.meshColor("green")
a.visitCommand('MeshAtts.pointSizePixels=10')
a.visitCommand('SetPlotOptions(MeshAtts)')
a.addField("PAR_VEF_P0P1_g_muscl/PAR_VEF_P0P1_g_muscl.lata","Mesh","dom_boundaries/Inlet")
a.plot()


### Physical properties of the fluid


In [ ]:
from trustutils import plot

a = plot.Table([""])
data = plot.loadText("PAR_VEF_P0P1_g_muscl/propertiesGeometry.dat")
a.addLine([data[1]],r'Density $\rho$ $[kg/m^3]$')
a.addLine([data[0]],r'Dynamic viscosity $\mu$ $[kg/(m.s)]$')
display(a)

### Flow characteristics


In [ ]:
a = plot.Table([""])
data = plot.loadText("PAR_VEF_P0P1_g_muscl/propertiesGeometry.dat")
a.addLine([data[2]],r'$u_{max}$')
a.addLine([data[3]],r'$u_{moy}$')
a.addLine([data[5]],r'Reynolds number $Re$')
a.addLine([data[4]],r'Theoretical volume flow rate $Q_v$')
a.addLine([data[6]],r'$\Delta P$ theoretical')
display(a)

### Initial conditions

Velocity is equal to 0 at the initial time

### Boundary conditions

    Inlet Frontiere_ouverte_vitesse_imposee Champ_front_fonc_xyz 3 (y^2+z^2<0.01)∗(−(0.01/0.1^2)∗(y^2+z^2−0.1^2))  0.  0.  **# Vmax 0 . 0 1 #**

    Outlet Frontiere_ouverte_pression_imposee Champ_front_Uniforme 1 1 0 .

    Wall paroi fixe

## Cylinder along z axis and $\vec{g}=-g \vec{e}_z$ 

### Initial state: Velocity field

We impose in inlet: $$u(r)=\frac{K}{4 \mu} \left(r^2-R^2\right)=-\frac{u_{max}}{R^2} \left(r^2-R^2\right)$$ with $u_{max}=u(0)=0.01$, $R=0.1$ and $r=y^2+z^2$

In [ ]:
a = visit.Show("PAR_VEF_P0P1_g_muscl/PAR_VEF_P0P1_g_muscl.lata","Pseudocolor","norme_VITESSE_FACES_dom_dual")
a.up3D([0, 1, -0.08])
a.normal3D([-0.65,0.1,0.75])
a.zoom3D([0.15,0.15,0.8])
a.plot()

### Initial state: Pressure field

The imposed pressure in boundary condition correspond to the TRUST field named 'Pressure'.This is the Navier-Stokes pressure $P_{NS}$ in Pascals (for incompressible flow):

   **$P_{NS}$ ='Pression\_Pa'='Pressure' in [Pa]**.

In [ ]:
a = visit.Show("PAR_VEF_P0P1_g_muscl/PAR_VEF_P0P1_g_muscl.lata","Pseudocolor","PRESSION_PA_ELEM_dom",iteration=0,min=0.01,max=10)
a.up3D([0, 1, -0.08])
a.normal3D([-0.65,0.1,0.75])
a.zoom3D([0.15,0.15,0.8])
a.plot()

and: **'Pression' = 'P\_star' = $\frac{1}{\rho} P_{NS}$ in $[m^2/s^2]$**. 

In [ ]:
a = visit.Show("PAR_VEF_P0P1_g_muscl/PAR_VEF_P0P1_g_muscl.lata","Pseudocolor","PRESSION_ELEM_dom",iteration=0,min=0.01,max=10)
a.up3D([0, 1, -0.08])
a.normal3D([-0.65,0.1,0.75])
a.zoom3D([0.15,0.15,0.8])
a.plot()

### Mean of the pressure field at the Outlet

We want to verify the pressure at the outlet of the cylinder where we imposed the pressure field to be 10. For that we create a new field with the 'Definition_champ' TRUST keyword. 

Syntax of the Definition_champ block for the mean of the 'pression' field on the outlet in the data

    mean_PRESSION_Outlet Reduction_0D {
      methode average
      source interpolation {
          domaine dom_boundaries_Outlet
          localisation elem
          source refChamp { Pb_champ pb pression }
      }
    }

The syntax for the probe:

    mean_PRESSION_Outlet mean_PRESSION_Outlet periode 5 numero_elem_sur_maitre 0

The obtained values confirm that Navier-stokes pressure 

**$P_{NS}$ ='Pression\_Pa'='Pressure'** in [Pa] and **'Pression' = 'P\_star' = $\frac{1}{\rho} P_{NS}$** in $[m^2/s^2]$

In [ ]:
from trustutils import plot

a = plot.Table([""])
data = plot.loadText("PAR_VEF_P0P1_g_muscl/mean_pressure.dat")
a.addLine([data[1]],r"Mean 'Pression_Pa' field in Pascals")
a.addLine([data[2]],r"Mean 'Pressure' field in Pascals")
a.addLine([data[0]],r"Mean 'Pression' field in $[m^2/s^2]$")
a.addLine([data[3]],r"Mean 'P_Star' field in $[m^2/s^2]$")
display(a)

### Flow rate

In [ ]:
from trustutils import plot

a = plot.Table(["Theoretical volume flow rate","TRUST volume flow rate",r"Error on $Q_v$ in $\%$"])

data = plot.loadText("PAR_VEF_P0P1_g_muscl/propertiesGeometry.dat")
a.addLine([[data[4],data[15],data[16]]],"muscl")

data = plot.loadText("PAR_VEF_P0P1_g_ef_stab/propertiesGeometry.dat")
a.addLine([[data[4],data[15],data[16]]],"Ef_Stab")

data = plot.loadText("PAR_VEF_P0P1_g_negligeable/propertiesGeometry.dat")
a.addLine([[data[4],data[15],data[16]]],"Negligeable")

display(a)

<!-- ### Velocity slice belong Y axis -->

### Velocity profile for a Poiseuille flow at final time step at $x=0.05$

    vitX vitesseX periode 20 segment 22    0.01 0.0 −0.1    0.01 0.0 0.1

In [ ]:
import numpy as np

a = plot.Graph(r"Velocity profil for a Poiseuille flow at final time step for $x=0.05$")
x = np.linspace(-0.1,0.1,100)
y = -0.01*(x*x-0.1*0.1)/(0.1*0.1)
a.add(x,y,label='Analytic solution')
a.addSegment("PAR_VEF_P0P1_g_muscl/PAR_VEF_P0P1_g_muscl_VITX.son",compo=0,label="Muscl vitX probe",marker="-x")
a.addSegment("PAR_VEF_P0P1_g_ef_stab/PAR_VEF_P0P1_g_ef_stab_VITX.son",compo=0,label="Ef_stab vitX probe",marker="-*")
a.addSegment("PAR_VEF_P0P1_g_negligeable/PAR_VEF_P0P1_g_negligeable_VITX.son",compo=0,label="Negligeable vitX probe",marker='-s',mfc='None')
a.label("X (m)","Velocity (m/s)")

<!-- ### Pressure slice belong Y axis -->

### Total pressure profile

Here we don't concider the hydrostatic pressure because in this configuration, the gravity pushes the flow.

    energie_cine_elem Predefini { Pb_champ pb energie_cinetique_elem }
    pressure_tot Transformation {
        methode formule
        expression 1 p_pa+p_hydro+energie_cine_elem
        localisation elem
        sources_reference { energie_cine_elem }
        sources {
            RefChamp { Pb_champ pb pression_pa nom_source p_pa } ,
            RefChamp { Pb_champ pb masse_volumique nom_source rho } ,
            RefChamp { Pb_champ pb pression_hydrostatique nom_source p_hydro }
        }
    }

In [ ]:
from trustutils import plot

a = plot.Graph("Totale pressure profile")

a.addSegment("PAR_VEF_P0P1_g_muscl/PAR_VEF_P0P1_g_muscl_PTOT.son",label=r"Muscl $P_{tot}$ probe",marker="-x")
a.addSegment("PAR_VEF_P0P1_g_ef_stab/PAR_VEF_P0P1_g_ef_stab_PTOT.son",label=r"Ef_stab $P_{tot}$ probe",marker="-*")
a.addSegment("PAR_VEF_P0P1_g_negligeable/PAR_VEF_P0P1_g_negligeable_PTOT.son",label=r"Negligeable $P_{tot}$ probe",marker="-s",mfc='None')
a.label("X (m)","pressure (Pa)")

### Pressure profile

In [ ]:
a = plot.Graph("Pressure profile")
x = np.linspace(0,0.1,100)
y = -4*0.001002*0.01*(x-0.1)/(0.1*0.1)+10
a.add(x,y,label='Analytic solution')
a.addSegment("PAR_VEF_P0P1_g_muscl/PAR_VEF_P0P1_g_muscl_P_PA.son",label="Muscl P PA probe",marker='-x')
a.addSegment("PAR_VEF_P0P1_g_ef_stab/PAR_VEF_P0P1_g_ef_stab_P_PA.son",label="Ef_stab P PA probe",marker="-*")
a.addSegment("PAR_VEF_P0P1_g_negligeable/PAR_VEF_P0P1_g_negligeable_P_PA.son",label="Negligeable P PA probe",marker="-s",mfc='None')
a.label("X (m)","pressure (Pa)")

### Hydrostatic pressure profile

In [ ]:
a = plot.Graph("Pressure hydrostatic profile")
x = np.linspace(0,0.1,100)
y = 0.*x
a.add(x,y,label='Analytic solution')
a.addSegment("PAR_VEF_P0P1_g_muscl/PAR_VEF_P0P1_g_muscl_PRESSION_HYDRO.son",label="Muscl pression hydro probe",marker='-x')
a.addSegment("PAR_VEF_P0P1_g_ef_stab/PAR_VEF_P0P1_g_ef_stab_PRESSION_HYDRO.son",label="Ef_stab pression hydro probe",marker="-*")
a.addSegment("PAR_VEF_P0P1_g_negligeable/PAR_VEF_P0P1_g_negligeable_PRESSION_HYDRO.son",label="Negligeable pression hydro probe",marker="-s",mfc='None')
a.label("X (m)","pressure (Pa)")

### Kinetic energy profile

In [ ]:
a = plot.Graph("Kinetic energy profile")
x = np.linspace(0,0.1,100)
y = [0.05]*100
a.add(x,y,label='Analytic solution')
a.addSegment("PAR_VEF_P0P1_g_muscl/PAR_VEF_P0P1_g_muscl_EC.son",label="Muscl ec probe",marker='-x')
a.addSegment("PAR_VEF_P0P1_g_ef_stab/PAR_VEF_P0P1_g_ef_stab_EC.son",label="Ef_stab ec probe",marker="-*")
a.addSegment("PAR_VEF_P0P1_g_negligeable/PAR_VEF_P0P1_g_negligeable_EC.son",label="Negligeable ec probe",marker="-s",mfc='None')
a.label("X (m)","pressure (Pa)")

### Pressure drop on the mean of the pressure at the inlet and outlet

In [ ]:
from trustutils import plot

a = plot.Table([r"$\Delta P$ Theoretical",r"$\Delta P$ TRUST",r"Error on $\Delta P$ in $\%$"])

data = plot.loadText("PAR_VEF_P0P1_g_muscl/propertiesGeometry.dat")
a.addLine([[data[6],data[13],data[14]]],"muscl")

data = plot.loadText("PAR_VEF_P0P1_g_ef_stab/propertiesGeometry.dat")
a.addLine([[data[6],data[13],data[14]]],"Ef_Stab")

data = plot.loadText("PAR_VEF_P0P1_g_negligeable/propertiesGeometry.dat")
a.addLine([[data[6],data[13],data[14]]],"Negligeable")

display(a)

### Pressure drop between the two symmetrical points of the probe 1

In [ ]:
from trustutils import plot

a = plot.Table([r"$\Delta P$ Theoretical",r"$\Delta P$ TRUST",r"Error on $\Delta P$ in $\%$"])

data = plot.loadText("PAR_VEF_P0P1_g_muscl/propertiesGeometry.dat")
a.addLine([[data[6],data[7],data[8]]],"muscl")

data = plot.loadText("PAR_VEF_P0P1_g_ef_stab/propertiesGeometry.dat")
a.addLine([[data[6],data[7],data[8]]],"Ef_Stab")

data = plot.loadText("PAR_VEF_P0P1_g_negligeable/propertiesGeometry.dat")
a.addLine([[data[6],data[7],data[8]]],"Negligeable")

display(a)

### Pressure drop between the two symmetrical points of the probe 3

In [ ]:
from trustutils import plot

a = plot.Table([r"$\Delta P$ Theoretical",r"$\Delta P$ TRUST",r"Error on $\Delta P$ in $\%$"])

data = plot.loadText("PAR_VEF_P0P1_g_muscl/propertiesGeometry.dat")
a.addLine([[data[6],data[11],data[12]]],"muscl")

data = plot.loadText("PAR_VEF_P0P1_g_ef_stab/propertiesGeometry.dat")
a.addLine([[data[6],data[11],data[12]]],"Ef_Stab")

data = plot.loadText("PAR_VEF_P0P1_g_negligeable/propertiesGeometry.dat")
a.addLine([[data[6],data[11],data[12]]],"Negligeable")

display(a)

## Cylinder along x axis and $\vec{g}=-g \vec{e}_x$ 

### Initial state: Velocity field

We impose in inlet: $$u(r)=-\frac{u_{max}}{R^2} \left(r^2-R^2\right)$$ with $u_{max}=u(0)=0.01$, $R=0.1$ and $r=y^2+z^2$

In [ ]:
a = visit.Show("PAR_VEF_P0P1_gx_muscl/PAR_VEF_P0P1_gx_muscl.lata","Pseudocolor","norme_VITESSE_FACES_dom_dual")
a.up3D([0, 1, -0.08])
a.normal3D([-0.65,0.1,0.75])
a.zoom3D([0.15,0.15,0.8])
a.plot()

### Initial state: Pressure field

The imposed pressure in boundary condition correspond to the TRUST field named 'Pressure'.This is the Navier-Stokes pressure $P_{NS}$ in Pascals (for incompressible flow):

   **$P_{NS}$ ='Pression\_Pa'='Pressure' in [Pa]**.
   
<!--    and: **'Pression' = 'P\_star' = $\frac{1}{\rho} P_{NS}$ in $[m^2/s^2]$** -->

In [ ]:
a = visit.Show("PAR_VEF_P0P1_gx_muscl/PAR_VEF_P0P1_gx_muscl.lata","Pseudocolor","PRESSION_PA_ELEM_dom",iteration=0,min=0.01,max=10)
a.up3D([0, 1, -0.08])
a.normal3D([-0.65,0.1,0.75])
a.zoom3D([0.15,0.15,0.8])
a.plot()

and: **'Pression' = 'P\_star' = $\frac{1}{\rho} P_{NS}$ in $[m^2/s^2]$**. 

In [ ]:
a = visit.Show("PAR_VEF_P0P1_gx_muscl/PAR_VEF_P0P1_gx_muscl.lata","Pseudocolor","PRESSION_ELEM_dom",iteration=0,min=0.01,max=10)
a.up3D([0, 1, -0.08])
a.normal3D([-0.65,0.1,0.75])
a.zoom3D([0.15,0.15,0.8])
a.plot()

### Mean of the pressure field at the Outlet

We want to verify the pressure at the outlet of the cylinder where we imposed the pressure field to be 10. For that we create a new field with the 'Definition_champ' TRUST keyword. 

Syntax of the Definition_champ block for the mean of the 'pression' field on the outlet in the data


    mean_PRESSION_Outlet Reduction_0D {
        methode average
        source interpolation {
            domaine dom_boundaries_Outlet
            localisation elem
            source refChamp { Pb_champ pb pression }
        }
    }

The syntax of the probe in the datafile is:

    mean_PRESSION_Outlet mean_PRESSION_Outlet periode 5 numero_elem_sur_maitre 0

The obtained values confirm that Navier-stokes pressure 

**$P_{NS}$ ='Pression\_Pa'='Pressure'** in [Pa] and **'Pression' = 'P\_star' = $\frac{1}{\rho} P_{NS}$** in $[m^2/s^2]$ 

In [ ]:
from trustutils import plot

a = plot.Table([""])
data = plot.loadText("PAR_VEF_P0P1_gx_muscl/mean_pressure.dat")
a.addLine([data[1]],"Mean 'Pression_Pa' field in Pascals")
a.addLine([data[2]],"Mean 'Pressure' field in Pascals")
a.addLine([data[0]],r"Mean 'Pression' field in $[m^2/s^2]$")
a.addLine([data[3]],r"Mean 'P_Star' field in $[m^2/s^2]$")
display(a)

### Flow rate

In [ ]:
from trustutils import plot

a = plot.Table(["Theoretical volume flow rate","TRUST volume flow rate",r"Error on $Q_v$ in $\%$"])

data = plot.loadText("PAR_VEF_P0P1_gx_muscl/propertiesGeometry.dat")
a.addLine([[data[4],data[15],data[16]]],"muscl")

data = plot.loadText("PAR_VEF_P0P1_gx_ef_stab/propertiesGeometry.dat")
a.addLine([[data[4],data[15],data[16]]],"Ef_Stab")

data = plot.loadText("PAR_VEF_P0P1_gx_negligeable/propertiesGeometry.dat")
a.addLine([[data[4],data[15],data[16]]],"Negligeable")

display(a)

<!-- ### Velocity slice belong Y axis -->

### Velocity profile for a Poiseuille flow at final time step at $x=0.05$

<!-- We impose in inlet: $$u(r)=\frac{K}{4 \mu} \left(r^2-R^2\right)=-\frac{u_{max}}{R^2} \left(r^2-R^2\right)$$ with $u_{max}=u(0)=0.01$, $R=0.1$ and $r=y^2+z^2$. -->

    vitX vitesseX periode 20 segment 22   0.01 0.0 −0.1    0.01 0.0 0.1

In [ ]:
import numpy as np

a = plot.Graph("Velocity profile for a Poiseuille flow at final time step for $x=0.05$")
x = np.linspace(-0.1,0.1,100)
y = -0.01*(x*x-0.1*0.1)/(0.1*0.1)
a.add(x,y,label='Analytic solution')
a.addSegment("PAR_VEF_P0P1_gx_muscl/PAR_VEF_P0P1_gx_muscl_VITX.son",compo=0,label="Muscl vitX probe",marker="-x")
a.addSegment("PAR_VEF_P0P1_gx_ef_stab/PAR_VEF_P0P1_gx_ef_stab_VITX.son",compo=0,label="Ef_stab vitX probe",marker="-*")
a.addSegment("PAR_VEF_P0P1_gx_negligeable/PAR_VEF_P0P1_gx_negligeable_VITX.son",compo=0,label="Negligeable vitX probe",marker='-s',mfc='None')
a.label("X (m)","Velocity (m/s)")

<!-- ### Pressure slice belong Y axis -->

### Total pressure profile

    pressure_tot Transformation {
        methode formule
        expression 1 p_pa+p_hydro+energie_cine_elem
        localisation elem
        sources reference { energie_cine_elem }
        sources {
            RefChamp { Pb_champ pb pression_pa nom_source p_pa } ,
            RefChamp { Pb_champ pb masse_volumique nom_source rho } ,
            RefChamp { Pb_champ pb pression_hydrostatique nom_source p_hydro }
        }
    }

In [ ]:
import numpy as np

a = plot.Graph("Totale pressure profile")
a.addSegment("PAR_VEF_P0P1_gx_muscl/PAR_VEF_P0P1_gx_muscl_PTOT.son",label=r"Muscl $P_{tot}$ probe",marker="-x")
a.addSegment("PAR_VEF_P0P1_gx_ef_stab/PAR_VEF_P0P1_gx_ef_stab_PTOT.son",label=r"Ef_stab $P_{tot}$ probe",marker="-*")
a.addSegment("PAR_VEF_P0P1_gx_negligeable/PAR_VEF_P0P1_gx_negligeable_PTOT.son",label=r"Negligeable $P_{tot}$ probe",marker="-s",mfc='None')
a.label("X (m)","pressure (Pa)")

### Pressure profile

In [ ]:
a = plot.Graph("Pressure profile")
x = np.linspace(0,0.1,100)
y = -4*0.001002*0.01*(x-0.1)/(0.1*0.1)+10
a.add(x,y,label='Analytic solution')
a.addSegment("PAR_VEF_P0P1_gx_muscl/PAR_VEF_P0P1_gx_muscl_P_PA.son",label="Muscl P PA probe",marker='-x')
a.addSegment("PAR_VEF_P0P1_gx_ef_stab/PAR_VEF_P0P1_gx_ef_stab_P_PA.son",label="Ef_stab P PA probe",marker="-*")
a.addSegment("PAR_VEF_P0P1_gx_negligeable/PAR_VEF_P0P1_gx_negligeable_P_PA.son",label="Negligeable P PA probe",marker="-s",mfc='None')
a.label("X (m)","pressure (Pa)")

### Hydrostatic pressure profile

In [ ]:
a = plot.Graph("Hydrostatic pressure profile")
x = np.linspace(0,0.1,100)
y = -1000*9.81*x
a.add(x,y,label='Analytic solution')
a.addSegment("PAR_VEF_P0P1_gx_muscl/PAR_VEF_P0P1_gx_muscl_PRESSION_HYDRO.son",label="Muscl pression hydro probe",marker='-x')
a.addSegment("PAR_VEF_P0P1_gx_ef_stab/PAR_VEF_P0P1_gx_ef_stab_PRESSION_HYDRO.son",label="Ef_stab pression hydro probe",marker="-*")
a.addSegment("PAR_VEF_P0P1_gx_negligeable/PAR_VEF_P0P1_gx_negligeable_PRESSION_HYDRO.son",label="Negligeable pression hydro probe",marker="-s",mfc='None')
a.label("X (m)","pressure (Pa)")

### Kinetic energy profile

In [ ]:
a = plot.Graph("Kinetic energy profile")
x = np.linspace(0,0.1,100)
y = [0.05]*100
a.add(x,y,label='Analytic solution')
a.addSegment("PAR_VEF_P0P1_gx_muscl/PAR_VEF_P0P1_gx_muscl_EC.son",label="Muscl ec probe",marker='-x')
a.addSegment("PAR_VEF_P0P1_gx_ef_stab/PAR_VEF_P0P1_gx_ef_stab_EC.son",label="Ef_stab ec probe",marker="-*")
a.addSegment("PAR_VEF_P0P1_gx_negligeable/PAR_VEF_P0P1_gx_negligeable_EC.son",label="Negligeable ec probe",marker="-s",mfc='None')
a.label("X (m)","pressure (Pa)")

### Pressure drop on the mean of the pressure at the inlet and outlet

In [ ]:
from trustutils import plot

a = plot.Table([r"$\Delta P$ Theoretical",r"$\Delta P$ TRUST",r"Error on $\Delta P$ in $\%$"])

data = plot.loadText("PAR_VEF_P0P1_gx_muscl/propertiesGeometry.dat")
a.addLine([[data[6],data[13],data[14]]],"muscl")

data = plot.loadText("PAR_VEF_P0P1_gx_ef_stab/propertiesGeometry.dat")
a.addLine([[data[6],data[13],data[14]]],"Ef_Stab")

data = plot.loadText("PAR_VEF_P0P1_gx_negligeable/propertiesGeometry.dat")
a.addLine([[data[6],data[13],data[14]]],"Negligeable")

display(a)

### Pressure drop between the two symmetrical points of the probe 1

In [ ]:
from trustutils import plot

a = plot.Table([r"$\Delta P$ Theoretical",r"$\Delta P$ TRUST",r"Error on $\Delta P$ in $\%$"])

data = plot.loadText("PAR_VEF_P0P1_gx_muscl/propertiesGeometry.dat")
a.addLine([[data[6],data[7],data[8]]],"muscl")

data = plot.loadText("PAR_VEF_P0P1_gx_ef_stab/propertiesGeometry.dat")
a.addLine([[data[6],data[7],data[8]]],"Ef_Stab")

data = plot.loadText("PAR_VEF_P0P1_gx_negligeable/propertiesGeometry.dat")
a.addLine([[data[6],data[7],data[8]]],"Negligeable")

display(a)

### Pressure drop between the two symmetrical points of the probe 3

In [ ]:
from trustutils import plot

a = plot.Table([r"$\Delta P$ Theoretical",r"$\Delta P$ TRUST",r"Error on $\Delta P$ in $\%$"])

data = plot.loadText("PAR_VEF_P0P1_gx_muscl/propertiesGeometry.dat")
a.addLine([[data[6],data[11],data[12]]],"muscl")

data = plot.loadText("PAR_VEF_P0P1_gx_ef_stab/propertiesGeometry.dat")
a.addLine([[data[6],data[11],data[12]]],"Ef_Stab")

data = plot.loadText("PAR_VEF_P0P1_gx_negligeable/propertiesGeometry.dat")
a.addLine([[data[6],data[11],data[12]]],"Negligeable")

display(a)

## Data Files

### PAR_VEF_P0P1_g_muscl

In [ ]:
run.dumpDataset("PAR_VEF_P0P1_g_muscl/PAR_VEF_P0P1_g_muscl")